In [2]:
import numpy as np
import pandas as pd
import cv2
import pytesseract

import os
from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [3]:
imgPaths = glob('../tvrtke/*.jpg')

In [10]:
allCompanies = pd.DataFrame(columns=['id','text'])
for imgPath in tqdm(imgPaths, desc='All Companies From Images'):
    #imgPath = imgPaths[0]
    _, filename = os.path.split(imgPath)

    #print(imgPath)
    #Extract data and text
    image = cv2.imread(imgPath)
    data = pytesseract.image_to_data(image, lang='hrv')

    #Dataframe
    dataList = list(map(lambda x: x.split('\t'), data.split('\n')))
    df = pd.DataFrame(dataList[1:], columns = dataList[0])
    df.dropna(inplace=True)

    #conf - from object into float
    df['conf'] = df['conf'].astype(float)

    # selecting only Level 5 data
    usefulData = df.query('conf >= 30')
    
    # creating new DataFrame
    company = pd.DataFrame()
    company['text'] = usefulData['text']
    company['id'] = filename
    
    # add company into allCompanies
    allCompanies = pd.concat((allCompanies, company))

All Companies From Images: 100%|█████████████████████████████████████████████████████| 401/401 [02:39<00:00,  2.51it/s]


# Saving data into csv file

In [11]:
allCompanies


,id,text
4,19.jpg,
8,19.jpg,6.
9,19.jpg,MAJ
13,19.jpg,GRAFIČKO
14,19.jpg,—
...,...,...
52,zzConcept.JPG,2100045
54,zzConcept.JPG,MBS:
55,zzConcept.JPG,080558280
57,zzConcept.JPG,OIB:


In [15]:
allCompanies.to_csv('tvrtke3.csv', index=False, encoding = 'utf-8-sig')